In [13]:
import numpy as np
import math
import time

# Function to perform bit-reversal (duh...)
def bitReverse(num, logn):
    rev_num = 0
    for i in range(logn):
        if (num >> i) & 1:
            rev_num |= 1 << (logn - 1 - i)
    return rev_num

# Barret Reduction
def barret_reduction(c1):
    p = 0
    m = 5039
    k = 24
    t123 = m * c1
    t = t123 // 2**24
    ta = q * t
    c = c1 - ta
    if c > q:
        c = c - q
    return c

def butterfly_dit(w, u, v):
    v1 = barret_reduction(w*v)
    x = (u + v1) % q
    y = (u - v1) % q
    return [x, y]

def butterfly_dif(w, u, v):
    x = (u + v) % q
    y1 = (u - v) % q
    y = barret_reduction(w*y1)
    return [x, y]

# Function to generate twiddle factors (for both forward and inverse NTT)
def gen_tf(psin, inv_psin, n, q):
    positions = [bitReverse(x, int(np.log2(n))) for x in range(n)]
    # print("DEBUGGGGGGGGGGGGGGGGGGGG")
    # print(positions)
    tmp1, tmp2 = [], []
    psis, inv_psis = [], []
    psi = 1
    inv_psi = 1
    for x in range(n):
        tmp1.append(psi)
        tmp2.append(inv_psi)
        psi = psi * psin % q
        inv_psi = inv_psi * inv_psin % q
    for x in range(n):
        val = tmp1[positions[x]]
        inv_val = tmp2[positions[x]]
        psis.append(val)
        inv_psis.append(inv_val)
# Convert lists to NumPy arrays
    psis = np.array(psis)
    inv_psis = np.array(inv_psis)
    return psis, inv_psis

# Function to generate scaling factors for point wise multiplication
def gen_pwmf(psin, n, q):
    pwmf = []
    for i in range(n):
        val = (psin**(2*bitReverse(i, int(np.log2(n))) + 1)) % q
        pwmf.append(val)
    pwmf = np.array(pwmf) 
    return pwmf

# Address generator for NTT and INTT
def addr_gen(s, i, l, v):
    j = s >> (l - 1 - i)
    k = s & ((v >> i) - 1)
    ie_r = j * (1 << (l - i)) + k
    io_r = ie_r + (1 << (l - i - 1))
    iw = (1 << i) + (s >> (l - i - 1))
    return ie_r, io_r, iw

# 128 point NTT using Cooley-Tucky Butterfly
def ct_ntt(a, psis, q, n):
    l = int(np.log2(n))
    v = n // 2
    for i in range(l): # l = 7
        for s in range(v): # v = 64
            ie, io, iw = addr_gen(s, i, l, v)
            S = psis[iw]
            U = a[ie]
            V = a[io]
            X = butterfly_dit(S, U, V)
            a[ie] = X[0]
            a[io] = X[1]
    return a

# 128 point INTT using Gentleman-Sande Butterfly
def gs_intt(a, inv_psis, q, n, inv_n):
    l = int(np.log2(n))
    v = n // 2
    for i in range(l): # l = 7
        for s in range(v): # v = 64
            i2 = l - i - 1
            ie, io, iw = addr_gen(s, i2, l, v)
            S = inv_psis[iw]
            U = a[ie]
            V = a[io]
            X = butterfly_dif(S, U, V)
            a[ie] = X[0]
            a[io] = X[1]
    for i in range(n):
        a[i] = a[i] * inv_n % q
    return a


# 256-point NTT using two 128-point NTTs
def ntt_256(x, psis, q, n):
    xe, xo = [], []
    for i in range(n2):
        if i%2 == 0:
            xe.append(x[i])
        else:
            xo.append(x[i])
    ye = ct_ntt(xe, psis, q, n)
    yo = ct_ntt(xo, psis, q, n)
    y = list(ye) + list(yo)
    return y

# 256-point INTT using two 128-point INTTs
def intt_256(y, inv_psis, q, n, inv_n):
    ye = y[0:128]
    yo = y[128:256]
    ze = gs_intt(ye, inv_psis, q, n, inv_n)
    zo = gs_intt(yo, inv_psis, q, n, inv_n)
    z = []
    for i in range(n):
        z.append(ze[i])
        z.append(zo[i])
    return z

# Point Wise Multiplication
def point_wise_mult(y1, y2, pwmf):
    # #print the input arrays
    # #print("Input array y1:")
    # #print(y1)
    # #print("\nInput array y2:")
    # #print(y2)
    # #print("\nInput array pwmf:")
    # #print(pwmf)
    y1e = y1[0:128]
    y1o = y1[128:256]
    y2e = y2[0:128]
    y2o = y2[128:256]
    y3e, y3o = [], []
    for i in range(n):
        y3e.append(((y1e[i] * y2e[i]) % q + (((y1o[i] * y2o[i]) % q) * pwmf[i]) % q) % q)
        y3o.append(((y1e[i] * y2o[i]) % q + (y1o[i] * y2e[i]) % q) % q)
    y3 = y3e + y3o
    
    # #print("Y3 Array inside PWM function: \n")
    # num_elements = len(y3)
    # #print("Number of elements:", num_elements)
    # #print(y3)
    return y3s



In [14]:
import time #To check execution time

import numpy as np
from numpy.polynomial import polynomial as poly



def polymul(x, y, modulus, poly_mod):
    """Add two polynoms
    Args:
        x, y: two polynoms to be added.
        modulus: coefficient modulus.
        poly_mod: polynomial modulus.
    Returns:
        A polynomial in Z_modulus[X]/(poly_mod).
    """
    return np.int64(
        np.round(poly.polydiv(poly.polymul(x, y) % modulus, poly_mod)[1] % modulus)
    )

def polyadd(x, y, modulus, poly_mod):
    """Multiply two polynoms
    Args:
        x, y: two polynoms to be multiplied.
        modulus: coefficient modulus.
        poly_mod: polynomial modulus.
    Returns:
        A polynomial in Z_modulus[X]/(poly_mod).
    """
    return np.int64(
        np.round(poly.polydiv(poly.polyadd(x, y) % modulus, poly_mod)[1] % modulus)
    )

In [15]:

def polymul_ntt(x, y, psis, inv_psis, q, n, poly_mod):
    """
    Multiply two polynomials using NTT.

    Args:
        x, y: Input polynomials.
        psis: Twiddle factors for NTT.
        inv_psis: Twiddle factors for INTT.
        q: Modulus.
        n: Polynomial size.
        poly_mod: Polynomial modulus (used for reduction if necessary).
    Returns:
        The product of two polynomials, reduced modulo `poly_mod`.
    """
    # Step 1: Forward NTT of both polynomials
    x_ntt = ntt_256(x, psis, q, n)
    y_ntt = ntt_256(y, psis, q, n)

    # Step 2: Point-wise multiplication
    pwmf = gen_pwmf(psis, n, q)
    result_ntt = point_wise_mult(x_ntt, y_ntt, pwmf)

    # Step 3: Inverse NTT to get the result back in the standard domain
    result = intt_256(result_ntt, inv_psis, q, n, inv_n=n)

    # Step 4: Reduce the result modulo the polynomial modulus (if required)
    result = np.polydiv(result % q, poly_mod)[1] % q

    return np.int64(result)

## Binary Polynomial 

The binary coefficients (0 or 1) define the polynomial. In this example, the polynomial is 

\[ P(x) = x^4 + x^2 + x + 1 \]

which means:

\[ 
P(x) = 1 \cdot x^4 + 0 \cdot x^3 + 1 \cdot x^2 + 1 \cdot x + 1 
\]

## Uniform Polynomial

The coefficients are uniformly sampled integers between 0 and the specified modulus (7 in this case). Here, the polynomial is:

\[ 
P(x) = 6x^4 + 5x^3 + 3x^2 + 4x + 2 
\]

## Normal Polynomial

The coefficients are sampled from a normal distribution and then rounded to integers. The result might include negative coefficients:

\[ 
P(x) = 3x^4 + 2x^3 - x + 1 
\]


In [16]:
def gen_binary_poly(size):
    return np.ones(size, dtype=np.int64)  # Return an array of all 1s, for example.
def gen_uniform_poly(size, modulus):
    return np.arange(size) % modulus  # Return a deterministic array using modulus.
def gen_normal_poly(size):
    return np.full(size, 2, dtype=np.int64)  # Return an array with all values set to 2.

In [17]:
# def gen_binary_poly(size):
#     return np.random.randint(0, 2, size, dtype=np.int64)

# def gen_uniform_poly(size, modulus):
#     return np.random.randint(0, modulus, size, dtype=np.int64)

# def gen_normal_poly(size):
#     return np.int64(np.random.normal(0, 2, size=size)) #Used to generate random numbers from a normal (or Gaussian) distribution.


def poly_to_string(coeffs):
    """
    Converts a polynomial (in coefficient form) to its string representation.
    Args:
        coeffs: List or array of polynomial coefficients, where coeff[i]
                is the coefficient of x^i.
    Returns:
        String representation of the polynomial.
    """
    terms = []
    degree = len(coeffs) - 1
    for i, coeff in enumerate(coeffs):
        if coeff == 0:
            continue
        elif i == 0:
            terms.append(f"{coeff}")
        elif i == 1:
            terms.append(f"{coeff}x")
        else:
            terms.append(f"{coeff}x^{i}")
    return " + ".join(terms[::-1]) if terms else "0"

# Example usage:
size = 5
modulus = 7  # For the uniform polynomial
binary_poly = gen_binary_poly(size)
uniform_poly = gen_uniform_poly(size, modulus)
normal_poly = gen_normal_poly(size)

print("Binary Polynomial:", poly_to_string(binary_poly))
print("Uniform Polynomial:", poly_to_string(uniform_poly))
print("Normal Polynomial:", poly_to_string(normal_poly))

Binary Polynomial: 1x^4 + 1x^3 + 1x^2 + 1x + 1
Uniform Polynomial: 4x^4 + 3x^3 + 2x^2 + 1x
Normal Polynomial: 2x^4 + 2x^3 + 2x^2 + 2x + 2



- Binary polynomials (like \(sk\)) are used because of their simplicity and security properties.
- Uniform polynomials (like \(a\)) add randomness to the public key, making the encryption process secure.
- Error polynomials (like \(e\)) introduce noise to the encryption, ensuring that even if part of the public key is known, the underlying secret key remains difficult to determine.
- The operations on the polynomials (addition and multiplication under moduli) ensure that the cryptographic system operates over a finite ring, which is crucial for security.


# Key Generation

The function returns the public key \((b, a)\) and the secret key \(sk\).

### Public Key

The public key is represented as:\[(b, a)\]

- Both polynomials \(b\) and \(a\) are used in the encryption process.
- Where, $b = (a \;. \;sk\; +\; e) (mod \; poly\_mod)$

### Secret Key

The secret key is represented as:\[sk\]

- This key is used for decryption.

In [18]:
def keygen(size, modulus, poly_mod):
    """Generate a public and secret keys
    Args:
        size: size of the polynoms for the public and secret keys.
        modulus: coefficient modulus.
        poly_mod: polynomial modulus.
    Returns:
        Public and secret key.
    """
    sk = gen_binary_poly(size)  #Secret Key
    a = gen_uniform_poly(size, modulus) #
    e = gen_normal_poly(size)
    temp = polymul(-a, sk, modulus, poly_mod)
    print("Poly mult result with Poly library")
    print(temp)

    temp = polymul_ntt(-a, sk, psis, inv_psis, q, n, poly_mod)
    print("Poly mult result with NTT")
    print(temp)

    b = polyadd(temp, -e, modulus, poly_mod)
    return (b, a), sk

In [19]:
# def keygen(size, modulus, poly_mod):
#     """Generate a public and secret keys
#     Args:
#         size: size of the polynoms for the public and secret keys.
#         modulus: coefficient modulus.
#         poly_mod: polynomial modulus.
#     Returns:
#         Public and secret key.
#     """
#     sk = gen_binary_poly(size)  #Secret Key
#     a = gen_uniform_poly(size, modulus) #
#     e = gen_normal_poly(size)
#     temp = polymul(-a, sk, modulus, poly_mod)
#     print(temp)
#     b = polyadd(temp, -e, modulus, poly_mod)
#     return (b, a), sk

In [20]:
def encrypt(pk, size, q, t, poly_mod, pt):
    """Encrypt an integer.
    Args:
        pk: public-key.
        size: size of polynomials.
        q: ciphertext modulus.
        t: plaintext modulus.
        poly_mod: polynomial modulus.
        pt: integer to be encrypted.
    Returns:
        Tuple representing a ciphertext.      
    """
    # encode the integer into a plaintext polynomial
    m = np.array([pt] + [0] * (size - 1), dtype=np.int64) % t
    delta = q // t
    scaled_m = delta * m  % q
    e1 = gen_normal_poly(size)
    e2 = gen_normal_poly(size)
    u = gen_binary_poly(size)
    ct0 = polyadd(
            polyadd(
                polymul(pk[0], u, q, poly_mod),
                e1, q, poly_mod),
            scaled_m, q, poly_mod
        )
    ct1 = polyadd(
            polymul(pk[1], u, q, poly_mod),
            e2, q, poly_mod
        )
    return (ct0, ct1)

In [21]:
def decrypt(sk, size, q, t, poly_mod, ct):
    """Decrypt a ciphertext
    Args:
        sk: secret-key.
        size: size of polynomials.
        q: ciphertext modulus.
        t: plaintext modulus.
        poly_mod: polynomial modulus.
        ct: ciphertext.
    Returns:
        Integer representing the plaintext.
    """
    scaled_pt = polyadd(
            polymul(ct[1], sk, q, poly_mod),
            ct[0], q, poly_mod
        )
    decrypted_poly = np.round(scaled_pt * t / q) % t
    return int(decrypted_poly[0])

In [22]:
def add_plain(ct, pt, q, t, poly_mod):
    """Add a ciphertext and a plaintext.
    Args:
        ct: ciphertext.
        pt: integer to add.
        q: ciphertext modulus.
        t: plaintext modulus.
        poly_mod: polynomial modulus.
    Returns:
        Tuple representing a ciphertext.
    """
    size = len(poly_mod) - 1
    # encode the integer into a plaintext polynomial
    m = np.array([pt] + [0] * (size - 1), dtype=np.int64) % t
    delta = q // t
    scaled_m = delta * m  % q
    new_ct0 = polyadd(ct[0], scaled_m, q, poly_mod)
    return (new_ct0, ct[1])

In [23]:
def mul_plain(ct, pt, q, t, poly_mod):
    """Multiply a ciphertext and a plaintext.
    Args:
        ct: ciphertext.
        pt: integer to multiply.
        q: ciphertext modulus.
        t: plaintext modulus.
        poly_mod: polynomial modulus.
    Returns:
        Tuple representing a ciphertext.
    """
    size = len(poly_mod) - 1
    # encode the integer into a plaintext polynomial
    m = np.array([pt] + [0] * (size - 1), dtype=np.int64) % t
    new_c0 = polymul(ct[0], m, q, poly_mod)
    new_c1 = polymul(ct[1], m, q, poly_mod)
    return (new_c0, new_c1)

In [29]:
# Scheme's parameters
# polynomial modulus degree
n = 2**8
# ciphertext modulus
q = 2**18
# plaintext modulus
t = 2**8
psin = 17
inv_psin = 1175
# polynomial modulus
poly_mod = np.array([1] + [0] * (n - 1) + [1])
n2 = 256

# Get pre-computed factors
psis, inv_psis = gen_tf(psin, inv_psin, n, q)
#print("PSIS:", psis)
# #print_2d_array(psis)
# #print("INV_PSIS:", inv_psis)

pwmf = gen_pwmf(psin, n, q)
# #print("PWMF:", pwmf)



# Keygen
pk, sk = keygen(n, q, poly_mod)

# Encryption
pt1, pt2 , pt3 = 73, 20, 10
cst1, cst2, cst3 = 7, 5, 10
ct1 = encrypt(pk, n, q, t, poly_mod, pt1)
ct2 = encrypt(pk, n, q, t, poly_mod, pt2)
ct3 = encrypt(pk, n, q, t, poly_mod, pt3)

# print("[+] Ciphertext ct1({}):".format(pt1))
# print("\n")
# print("\t ct1_0:", ct1[0])
# print("\t ct1_1:", ct1[1])
# print("\n")
# print("[+] Ciphertext ct2({}):".format(pt2))
# print("\n")
# print("\t ct1_0:", ct2[0])
# print("\t ct1_1:", ct2[1])
# print("\n")
# print("[+] Ciphertext ct1({}):".format(pt3))
# print("\n")
# print("\t ct1_0:", ct3[0])
# print("\t ct1_1:", ct3[1])
# print("\n")

# Evaluation
ct4 = add_plain(ct1, cst1, q, t, poly_mod)
ct5 = mul_plain(ct2, cst2, q, t, poly_mod)
ct6 = mul_plain(ct3, cst3, q, t, poly_mod)

# Decryption
d_start_ct4_time = time.time() 
decrypted_ct4 = decrypt(sk, n, q, t, poly_mod, ct4)
d_end_ct4_time = time.time()
print("[+] Decrypted ct4(ct1 + {}): {}".format(cst1, decrypted_ct4))
print(f"Decrypt time::{(d_end_ct4_time - d_start_ct4_time)*1000} mili seconds \n")

d_start_ct4_time = time.time() 
decrypted_ct5 = decrypt(sk, n, q, t, poly_mod, ct5)
d_end_ct4_time = time.time()
print("[+] Decrypted ct5(ct2 * {}): {}".format(cst2, decrypted_ct5))
print(f"Decrypt time::{(d_end_ct4_time - d_start_ct4_time)*1000} mili seconds \n")

d_start_ct4_time = time.time() 
decrypted_ct6 = decrypt(sk, n, q, t, poly_mod, ct6)
d_end_ct4_time = time.time()
print("[+] Decrypted ct6(ct3 * {}): {}".format(cst3, decrypted_ct6))
print(f"Decrypt time::{(d_end_ct4_time - d_start_ct4_time)*1000} mili seconds \n")

Poly mult result with Poly library
[ 32640  32638  32634  32628  32620  32610  32598  32584  32568  32550
  32530  32508  32484  32458  32430  32400  32368  32334  32298  32260
  32220  32178  32134  32088  32040  31990  31938  31884  31828  31770
  31710  31648  31584  31518  31450  31380  31308  31234  31158  31080
  31000  30918  30834  30748  30660  30570  30478  30384  30288  30190
  30090  29988  29884  29778  29670  29560  29448  29334  29218  29100
  28980  28858  28734  28608  28480  28350  28218  28084  27948  27810
  27670  27528  27384  27238  27090  26940  26788  26634  26478  26320
  26160  25998  25834  25668  25500  25330  25158  24984  24808  24630
  24450  24268  24084  23898  23710  23520  23328  23134  22938  22740
  22540  22338  22134  21928  21720  21510  21298  21084  20868  20650
  20430  20208  19984  19758  19530  19300  19068  18834  18598  18360
  18120  17878  17634  17388  17140  16890  16638  16384  16128  15870
  15610  15348  15084  14818  14550  14280

NameError: name 'n2' is not defined